# SomPyte — FAW CV5 (ResNet18/34)
Binary image classification for fall armyworm (AUC metric). This notebook is **self-contained** and produces three files:

- `submission_cv5_resnet18.csv`
- `submission_cv5_resnet34.csv`
- `submission_blend_r18_r34.csv`

**Time-safe:** early stopping on AUC, AMP, simple TTA (flip). **Rules-safe:** probabilities only, no external data, ImageNet weights.


In [ ]:

# %% [setup] Install minimal packages (Colab-safe; local may already have them)
# If running locally and you already have these, you can skip this cell.
import sys, subprocess

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].split('[')[0])
    except Exception:
        print(f"Installing: {pkg}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

for p in ["tqdm", "scikit-learn", "torchvision", "pandas", "numpy", "Pillow"]:
    pip_install(p)
print("✅ deps ready")


In [ ]:

# %% [imports]
import os, math, random, time, gc, json
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


In [ ]:

# %% [config]
SEED = 42
IMG_SIZE = 256
BATCH_SIZE = 48
EPOCHS = 12
N_FOLDS = 5
TTA = True  # horizontal flip only

# paths
DATA_DIR = Path("./")  # adjust if needed
TRAIN_CSV = DATA_DIR / "Train.csv"
TEST_CSV  = DATA_DIR / "Test.csv"
IMAGES_DIR = DATA_DIR / "Images"  # expect images in ./Images/<image_id>

assert TRAIN_CSV.exists(), "Train.csv not found"
assert TEST_CSV.exists(), "Test.csv not found"
assert IMAGES_DIR.exists(), "Images folder not found (unzip Images.zip -> Images/)"


In [ ]:

# %% [seed]
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)


In [ ]:

# %% [dataframe]
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)
print(df_train.shape, df_test.shape)
df_train.head()


In [ ]:

# %% [dataset]
class FAWDataset(Dataset):
    def __init__(self, df, root, train=True, transforms=None):
        self.df = df.reset_index(drop=True)
        self.root = Path(root)
        self.train = train
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / row["Image_id"]
        img = Image.open(img_path).convert("RGB")
        if self.transforms:
            img = self.transforms(img)
        if self.train:
            y = torch.tensor(row["Label"], dtype=torch.float32)
            return img, y
        else:
            return img, row["Image_id"]


In [ ]:

# %% [transforms]
train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(12),
    T.RandomPerspective(distortion_scale=0.15, p=0.25),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:

# %% [model]
def build_model(backbone="resnet18"):
    if backbone == "resnet18":
        m = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
    elif backbone == "resnet34":
        m = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
    else:
        raise ValueError("backbone must be resnet18 or resnet34")
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, 1)
    return m

class EMA:
    # simple EMA for model parameters
    def __init__(self, model, decay=0.999):
        self.decay = decay
        self.shadow = {}
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self, model):
        self.backup = {}
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data.clone()
                param.data = self.shadow[name]

    def restore(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad and name in self.backup:
                param.data = self.backup[name]
        self.backup = {}


In [ ]:

# %% [train utils]
def train_one_epoch(model, loader, optimizer, scaler):
    model.train()
    losses = []
    for imgs, y in loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True).unsqueeze(1)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="cuda", enabled=(DEVICE=="cuda")):
            logits = model(imgs)
            loss = F.binary_cross_entropy_with_logits(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        losses.append(loss.detach().item())
    return float(np.mean(losses))

@torch.no_grad()
def valid_one_epoch(model, loader):
    model.eval()
    probs, targs = [], []
    for imgs, y in loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        logits = model(imgs)
        p = torch.sigmoid(logits).squeeze(1).detach().cpu().numpy()
        probs.append(p)
        targs.append(y.numpy())
    probs = np.concatenate(probs)
    targs = np.concatenate(targs)
    auc = roc_auc_score(targs, probs)
    return auc, float(np.mean(probs))


In [ ]:

# %% [cv fit/predict]
def run_fold(df, fold, backbone="resnet18"):
    train_idx = df.index[df["kfold"] != fold]
    valid_idx = df.index[df["kfold"] == fold]
    dtr = df.loc[train_idx].reset_index(drop=True)
    dvl = df.loc[valid_idx].reset_index(drop=True)

    tr_ds = FAWDataset(dtr, IMAGES_DIR, train=True, transforms=train_tfms)
    vl_ds = FAWDataset(dvl, IMAGES_DIR, train=True, transforms=valid_tfms)
    te_ds = FAWDataset(df_test, IMAGES_DIR, train=False, transforms=valid_tfms)

    tr_ld = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    vl_ld = DataLoader(vl_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    te_ld = DataLoader(te_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    model = build_model(backbone).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    scaler = torch.amp.GradScaler(enabled=(DEVICE=="cuda"))
    ema = EMA(model, decay=0.999)

    best_auc, best_path = -1.0, f"best_{backbone}_fold{fold}.pt"

    for epoch in range(1, EPOCHS+1):
        tr_loss = train_one_epoch(model, tr_ld, optimizer, scaler)
        ema.update(model)
        # validate on EMA weights
        ema.apply_shadow(model)
        val_auc, _ = valid_one_epoch(model, vl_ld)
        ema.restore(model)
        scheduler.step()

        print(f"[{backbone}][fold {fold}][ep {epoch}] loss={tr_loss:.4f} AUC={val_auc:.6f}")
        if val_auc > best_auc:
            best_auc = val_auc
            torch.save(model.state_dict(), best_path)
            patience = 0
        else:
            patience += 1
            if patience >= 3:
                print("Early stop.")
                break

    # load best
    model.load_state_dict(torch.load(best_path, map_location=DEVICE))
    model.eval()

    # valid oof probs (EMA for final scoring)
    ema.apply_shadow(model)
    v_probs = []
    with torch.no_grad():
        for imgs, y in vl_ld:
            imgs = imgs.to(DEVICE, non_blocking=True)
            p = torch.sigmoid(model(imgs)).squeeze(1).detach().cpu().numpy()
            v_probs.append(p)
    ema.restore(model)
    v_probs = np.concatenate(v_probs)
    oof = pd.DataFrame({"idx": valid_idx, "oof": v_probs}).set_index("idx")

    # test predictions (+ optional TTA flip)
    te_probs = []
    with torch.no_grad():
        for (imgs, ids) in te_ld:
            imgs = imgs.to(DEVICE, non_blocking=True)
            p = torch.sigmoid(model(imgs)).squeeze(1)
            if TTA:
                p_flip = torch.sigmoid(model(torch.flip(imgs, dims=[3]))).squeeze(1)
                p = 0.5*(p + p_flip)
            te_probs.append(p.detach().cpu().numpy())
    te_probs = np.concatenate(te_probs)

    # cleanup
    del model, tr_ld, vl_ld, te_ld; gc.collect(); torch.cuda.empty_cache()
    return best_auc, oof, te_probs


In [ ]:

# %% [kfold split]
df = df_train.copy()
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
df["kfold"] = -1
for f, (_, val_idx) in enumerate(skf.split(df, df["Label"])):
    df.loc[val_idx, "kfold"] = f
df["kfold"].value_counts().sort_index()


In [ ]:

# %% [CV5 ResNet18]
backbone = "resnet18"
all_oof = np.zeros(len(df))
test_pred = np.zeros(len(df_test))

fold_aucs = []
for f in range(N_FOLDS):
    auc, oof, te = run_fold(df, f, backbone=backbone)
    fold_aucs.append(auc)
    all_oof[oof.index.values] = oof["oof"].values
    test_pred += te / N_FOLDS

print(f"{backbone} fold AUCs:", fold_aucs, "mean:", np.mean(fold_aucs))
oof_auc = roc_auc_score(df["Label"].values, all_oof)
print("OOF AUC:", oof_auc)

sub18 = pd.DataFrame({"Image_id": df_test["Image_id"], "Label": test_pred})
sub18.to_csv("submission_cv5_resnet18.csv", index=False)
print("Saved submission_cv5_resnet18.csv")


In [ ]:

# %% [CV5 ResNet34]
backbone = "resnet34"
all_oof = np.zeros(len(df))
test_pred = np.zeros(len(df_test))

fold_aucs = []
for f in range(N_FOLDS):
    auc, oof, te = run_fold(df, f, backbone=backbone)
    fold_aucs.append(auc)
    all_oof[oof.index.values] = oof["oof"].values
    test_pred += te / N_FOLDS

print(f"{backbone} fold AUCs:", fold_aucs, "mean:", np.mean(fold_aucs))
oof_auc = roc_auc_score(df["Label"].values, all_oof)
print("OOF AUC:", oof_auc)

sub34 = pd.DataFrame({"Image_id": df_test["Image_id"], "Label": test_pred})
sub34.to_csv("submission_cv5_resnet34.csv", index=False)
print("Saved submission_cv5_resnet34.csv")


In [ ]:

# %% [blend]
import pandas as pd
a = pd.read_csv("submission_cv5_resnet18.csv")
b = pd.read_csv("submission_cv5_resnet34.csv")
m = a.merge(b, on="Image_id", suffixes=("_r18","_r34"))
m["Label"] = 0.5*m["Label_r18"] + 0.5*m["Label_r34"]
m[["Image_id","Label"]].to_csv("submission_blend_r18_r34.csv", index=False)
print("Saved submission_blend_r18_r34.csv")
